# 3x3 gridworld with two agents

When multiple agents simultaneously interact, the environment is no longer stationary because each agent’s policy influences the environment dynamics. This setting is modeled as a general-sum stochastic game:


$\mathcal{G} = \langle S, (A_1, A_2), (r_1, r_2), p, \gamma \rangle$

where:

Each agent i has its own action space $A_i$.

Each agent receives a reward 
    $r_i$: $S \times A_1 \times A_2 \to \mathbb{R}$.

The state transition depends on both agents’ actions.

A major problem is how to define optimality when there are multiple agents.


In Nash Q-learning, agents do not maximize their own Q-values independently, but instead assume that both agents will play according to Nash equilibria at each stage of the game.

Instead of:


$Q(s, a) \leftarrow (1 - \alpha) Q(s, a) + \alpha \Big( r + \gamma \max_{a{\prime}} Q(s{\prime}, a{\prime}) \Big)$


it updates according to:


$Q_i(s, a_1, a_2) \leftarrow (1 - \alpha) Q_i(s, a_1, a_2) + \alpha \Big( r_i + \gamma \cdot \text{Nash}(Q(s{\prime})) \Big)$


where Nash(Q(s’)) represents the expected Q-value under a Nash equilibrium strategy.

For each state  s , a stage game is formed from the current Q-values:

A payoff matrix is constructed for each agent from $Q(s, a_1, a_2)$.

The Lemke-Howson algorithm is used to compute a Nash equilibrium strategy.

The Nash equilibrium expected values are used for Q-value updates.

In [ ]:
import random
import itertools
import pandas as pd

# possible actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

# state representation: 
# the state space is represented by a 9-element grid indexed from 0 to 8
# each state is a tuple: (agent1's position, agent2's position)
WORLD_HEIGHT = 3
WORLD_WIDTH = 3
gridIndexList = [WORLD_WIDTH * i + j for i in range(WORLD_HEIGHT) for j in range(WORLD_WIDTH)]

# valid moves are stored for each position to prevent illegal actions
locationValidActions = {i: [] for i in gridIndexList}
for i in range(WORLD_HEIGHT):
    for j in range(WORLD_WIDTH):
        gridIndexNumber = WORLD_WIDTH * i + j
        if i != WORLD_HEIGHT - 1:
            locationValidActions[gridIndexNumber].append(ACTION_UP)
        if i != 0:
            locationValidActions[gridIndexNumber].append(ACTION_DOWN)
        if j != 0:
            locationValidActions[gridIndexNumber].append(ACTION_LEFT)
        if j != WORLD_WIDTH - 1:
            locationValidActions[gridIndexNumber].append(ACTION_RIGHT)

# Define all possible states as (agent1_position, agent2_position)
statesAllOne = list(itertools.product(gridIndexList, repeat=2))

# each agent maintains:
# a Q-table storing joint action values
# a learning rate for updating values
# a policy strategy (not used now)
class Agent:
    def __init__(self, agentIndex=0, startLocationIndex=0):
        self.qTable = {}  # Stores Q-values
        self.alpha = {}  # Learning rates
        self.timeNumber = {}  # Number of visits
        self.strategy = {}  # Policy (not used in this experiment)
        self.agentIndex = agentIndex  # Identifies agent 0 or 1
        self.locationIndex = startLocationIndex  # Initial position

    def initialSelfQTable(self):
        """ Initialize Q-table with zero values for all state-action pairs """
        self.qTable[0] = {}
        self.qTable[1] = {}
        for state in statesAllOne:
            self.qTable[0][state] = {}
            self.qTable[1][state] = {}
            for action1 in locationValidActions[state[0]]:
                for action2 in locationValidActions[state[1]]:
                    self.qTable[0][state][(action1, action2)] = 0
                    self.qTable[1][state][(action1, action2)] = 0

    def initialSelfAlpha(self):
        """ Initialize learning rates """
        for state in statesAllOne:
            self.alpha[state] = 1.0  # initial learning rate

    def chooseActionRandomly(self, currentState):
        """ Choose a random valid action """
        return random.choice(locationValidActions[currentState[self.agentIndex]])

    def nashQLearning(self, gamma, action0, reward0, currentState, nextState, action1, reward1):
        """ Update Q-table using Nash Q-learning (simplified update) """
        alpha = self.alpha[currentState]
        self.qTable[0][currentState][(action0, action1)] += alpha * (
            reward0 + gamma * max(self.qTable[0][nextState].values()) - self.qTable[0][currentState][(action0, action1)]
        )
        self.qTable[1][currentState][(action0, action1)] += alpha * (
            reward1 + gamma * max(self.qTable[1][nextState].values()) - self.qTable[1][currentState][(action0, action1)]
        )

# Game environment
def gridGameOne(action0, action1, currentState):
    """ Simulates one step in the grid game """
    agent0_new = currentState[0]
    agent1_new = currentState[1]

    # Compute new positions based on actions
    if action0 == ACTION_UP and agent0_new + WORLD_WIDTH in gridIndexList:
        agent0_new += WORLD_WIDTH
    elif action0 == ACTION_DOWN and agent0_new - WORLD_WIDTH in gridIndexList:
        agent0_new -= WORLD_WIDTH
    elif action0 == ACTION_LEFT and (agent0_new - 1) in gridIndexList and (agent0_new % WORLD_WIDTH != 0):
        agent0_new -= 1
    elif action0 == ACTION_RIGHT and (agent0_new + 1) in gridIndexList and ((agent0_new + 1) % WORLD_WIDTH != 0):
        agent0_new += 1

    if action1 == ACTION_UP and agent1_new + WORLD_WIDTH in gridIndexList:
        agent1_new += WORLD_WIDTH
    elif action1 == ACTION_DOWN and agent1_new - WORLD_WIDTH in gridIndexList:
        agent1_new -= WORLD_WIDTH
    elif action1 == ACTION_LEFT and (agent1_new - 1) in gridIndexList and (agent1_new % WORLD_WIDTH != 0):
        agent1_new -= 1
    elif action1 == ACTION_RIGHT and (agent1_new + 1) in gridIndexList and ((agent1_new + 1) % WORLD_WIDTH != 0):
        agent1_new += 1

    nextState = (agent0_new, agent1_new)

    # Reward system (arbitrary for now)
    reward_0 = 1 if agent0_new == 8 else 0  # Reward if agent 0 reaches bottom-right
    reward_1 = 1 if agent1_new == 8 else 0  # Reward if agent 1 reaches bottom-right

    endGameFlag = agent0_new == 8 and agent1_new == 8  # game ends if both reach the goal

    return reward_0, reward_1, nextState, endGameFlag

# agents randomly explore actions 
# nash Q-learning updates are applied
def playGameOne():
    """ Runs the game loop for Nash Q-learning """
    gamma = 0.9
    agent_0 = Agent(0, random.choice(gridIndexList))
    agent_1 = Agent(1, random.choice(gridIndexList))

    agent_0.initialSelfQTable()
    agent_1.initialSelfQTable()
    agent_0.initialSelfAlpha()
    agent_1.initialSelfAlpha()

    episodes = 0
    history = []  # store history for visualization

    while episodes < 100:
        currentState = (agent_0.locationIndex, agent_1.locationIndex)

        while True:
            action0 = agent_0.chooseActionRandomly(currentState)
            action1 = agent_1.chooseActionRandomly(currentState)
            reward_0, reward_1, nextState, endGameFlag = gridGameOne(action0, action1, currentState)

            agent_0.nashQLearning(gamma, action0, reward_0, currentState, nextState, action1, reward_1)
            agent_1.nashQLearning(gamma, action0, reward_0, currentState, nextState, action1, reward_1)

            history.append((episodes, currentState, action0, action1, reward_0, reward_1, nextState))

            if endGameFlag:
                break

            currentState = nextState

        episodes += 1

    df_history = pd.DataFrame(history, columns=['Episode', 'State', 'Action0', 'Action1', 'Reward0', 'Reward1', 'NextState'])
    print(df_history) 
    #df_history.to_csv("game_history.csv") 

playGameOne()

      Episode   State  Action0  Action1  Reward0  Reward1 NextState
0           0  (4, 4)        3        0        0        0    (5, 7)
1           0  (5, 7)        2        1        0        0    (4, 4)
2           0  (4, 4)        1        3        0        0    (1, 5)
3           0  (1, 5)        3        0        0        1    (2, 8)
4           0  (2, 8)        0        1        0        0    (5, 5)
...       ...     ...      ...      ...      ...      ...       ...
7819       99  (5, 3)        1        0        0        0    (2, 6)
7820       99  (2, 6)        0        3        0        0    (5, 7)
7821       99  (5, 7)        1        3        0        1    (2, 8)
7822       99  (2, 8)        0        1        0        0    (5, 5)
7823       99  (5, 5)        0        0        1        1    (8, 8)

[7824 rows x 7 columns]


To do:

I have applied standard Q-Learning instead of Nash Q-learning --> Study the lemke-Howson algorithm and fix code (Instead of max(Q), use a Nash solver to find the equilibrium Q-values at nextState)

I keep alpha constant. This can prevent convergence. In Nash Q-learning, the learning rate decreases over time ($
\alpha(s, a_0, a_1) = \frac{1}{N(s, a_0, a_1)}
$) 

My agent policies are purely random (should I use an epsilon greedy policy?)